## INTRODUCTION

Après avoir nettoyé, exploré et vectorisé les données textuelles, cette section vise à appliquer des techniques de modélisation pour extraire de l'information ou prédire les tags associés aux questions Stack Overflow.

Nous distinguons deux approches complémentaires :

- **Modélisation non supervisée** : pour explorer les thématiques latentes du corpus sans utiliser les tags
- **Modélisation supervisée** : pour entraîner un modèle capable de prédire automatiquement les tags à partir du texte

Chaque approche sera testée sur les représentations vectorielles construites précédemment (TF-IDF, embeddings, CouterVectorizer), et évaluée selon des critères adaptés à la tâche.

Les modélisations seront réalisées dans deux notebooks séparés :
- `2_modelisation_non_supervisee.ipynb`
- `3_modelisation_supervisee.ipynb`



La modélisation supervisée vise à prédire automatiquement les tags associés à une question à partir de son texte.  
Il s’agit d’un problème de **classification multi-label**, où chaque observation (question) peut être associée à plusieurs classes (tags).

Dans cette sous-section, nous testons plusieurs modèles adaptés à cette tâche, tels que :

- **Logistic Regression** avec stratégie One-vs-Rest
- **Random Forest**, **SVM**, ou **XGBoost**
- **Réseaux de neurones** (optionnel)

Les performances seront évaluées à l’aide de métriques spécifiques au multi-label (F1-score, précision, rappel, etc.).

## 1. IMPORTS  

In [ ]:
# 🔧 Standard
import numpy as np
import pandas as pd

# 📊 Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# 🔬 Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, hamming_loss, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# 🧪 Traitement conditionnel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

# 💾 Persistance / outils externes
import joblib
import pickle
import os

# 🗃️ Utilitaires
from scipy.sparse import load_npz
from tqdm.notebook import tqdm  # ou simplement tqdm si console

# 💡 Tracking expérimental (optionnel)
import mlflow
import mlflow.sklearn  # Pour les modèles scikit-learn
import sys
import os

# Ajoute le dossier parent du notebook aux chemins Python
project_root = os.path.abspath("..")  # ou "../.." selon ton niveau
sys.path.append(project_root)
# 📁 Modules personnalisés
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)


## 2. MODELISATION : LOGISTIC REGRESSION

### 2.1. CHARGEMENT DES FEATURES

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)

# -----------------------------------------
# --- 0. CONFIGURATION DU MODÈLE À TESTER
# -----------------------------------------
# ✅ Tu peux changer ces lignes pour benchmarker un autre modèle
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model_type = "logreg"
model_class = LogisticRegression
model_wrapper = OneVsRestClassifier  # ou ClassifierChain ou None

# ------------------------------
# --- 1. SELECTION DES FEATURES
# ------------------------------
full_df = pd.read_parquet("data/processed/full_explo_wo.parquet")
print(f"Dimensions du dataframe full_df : {full_df.shape}")
print(f" Colonnes du dataframe full_df : {full_df.columns.tolist()}")
print(full_df[["clean_title_body"]].head(3))

# --- A. CHARGEMENT DES VECTEURS
X_bow   = load_npz("models/bow/X_bow_full.npz")
X_tfidf = load_npz("models/tfidf/X_tfidf_full.npz")
X_svd   = np.load("models/svd/X_titlebody_svd10k.npy")
X_w2v   = np.load("models/word2vec/X_w2v_full.npy")
X_use   = np.load("models/use_model/embeddings_use_full.npy")
X_sbert = np.load("models/sbert/embeddings_sbert_full.npy")

print("# --- DIMENSIONS DES VECTEURS :")
for name, mat in [("BoW", X_bow), ("TF-IDF", X_tfidf), ("SVD", X_svd),
                  ("Word2Vec", X_w2v), ("USE", X_use), ("SBERT", X_sbert)]:
    print(f"# --- {name:<10}: {mat.shape}")


# --- B. CHARGEMENT DES LABELS MULTILABEL
mlb = joblib.load("models/tags/multilabel_binarizer_full.pkl")
Y_full = np.load("models/tags/y_tags_full.npy")

print(f"# --- Labels multilabel chargés : {Y_full.shape}")
print(f"# --- Nombre de tags avant filtrage : {len(mlb.classes_)}")

tag_counts = Y_full.sum(axis=0)
tag_mask = tag_counts >= 1  # seuil de rareté
Y_full_filtered = Y_full[:, tag_mask]

mlb_filtered = mlb
mlb_filtered.classes_ = np.array(mlb.classes_)[tag_mask]
print(f"# --- TAGS conservés après filtrage : {len(mlb_filtered.classes_)}")
print(mlb_filtered.classes_.tolist())
print("java" in mlb_filtered.classes_)    # True ou False
print("python" in mlb_filtered.classes_)  # True ou False
tags_name = mlb.classes_
for tag in ["java", "python"]:
    idx = list(tags_name).index(tag)
    print(f"{tag} count: {tag_counts[idx]}")

from collections import Counter
import numpy as np
tag_freq = Counter({tag: tag_counts[i] for i, tag in enumerate(tags_name)})
print(tag_freq["asp.net-core-mvc"])  # → combien d’occurrences ?

# Top 10 tags les plus fréquents
top_tags = tag_freq.most_common(10)
print("🔝 Tags dominants :", top_tags)

### 2.2. DEFINITION DES VARIABLES VECTEURS A ENTRAINER

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ----------------------------------------------
# --- 2. SÉPARATION DES FEATURES ET DE LA CIBLE
# ----------------------------------------------
Y = Y_full_filtered
X_text = full_df["clean_title_body"]
print(f"# --- Matrice multilabel (Y) : {Y.shape}")
print(f"# --- Colonne textuelle (X) : {X_text.shape}")

# ---------------------------------------------------------
# --- 3. CRÉATION DES VECTEURS + DICTS DE SUPPORT
# ---------------------------------------------------------
X_dict = {
    "bow": X_bow,
    "tfidf": X_tfidf,
    "svd": X_svd,
    "w2v": X_w2v,
    "use": X_use,
    "sbert": X_sbert
}

preproc_dict = {
    "bow": None,
    "tfidf": None,
    "svd": None,
    "w2v": "scale",
    "use": "scale",
    "sbert": "scale"
}

### 2.3. DIVISION TRAIN/ TEST POUR CHAQUE VECTEUR

In [ ]:
# ---------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# ---------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 4. DIVISION EN TRAIN / TEST SUR CHAQUE VECTEUR
# ---------------------------------------------------------
indices = np.arange(Y.shape[0])
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

splits_dict = {}
for name, X in X_dict.items():
    X_train, X_test, y_train, y_test = mdl.split_on_indices(X, Y, train_idx, test_idx)
    splits_dict[name] = (X_train, X_test)

print(f"# --- Splits prêts pour vecteurs : {list(splits_dict.keys())}")


### 2.4. ENTRAINEMENT DU MODELE POUR CHAQUE VECTEURS - RESULTATS

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
import mlflow
import mlflow.sklearn

df_results = []
uri = "file:///D:/machine_learning_training/openclassrooms_projects/05_categorisez_automatiquement_question/mlruns"
mlflow.set_tracking_uri(uri)
mlflow.set_experiment("logreg_stackoverflow")


for name, (X_train, X_test) in splits_dict.items():
    preproc = preproc_dict.get(name)

    with mlflow.start_run(run_name=f"{model_type}_{name}"):
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("vecteur", name)
        mlflow.log_param("preprocessing", preproc)

        # 🧪 Entraînement + prédiction
        scores = mdl.train_and_score_vector_full_metrics(
            name=name,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            model_class=model_class,
            model_wrapper=model_wrapper,
            preprocess=preproc
        )

        # 📊 Logging des métriques
        mlflow.log_metric("f1_micro", scores["f1_micro"])
        mlflow.log_metric("hamming_loss", scores["hamming_loss"])
        mlflow.log_metric("coverage_score", scores["coverage_tags"])
        mlflow.log_metric("f1_macro", scores["f1_macro"])
        mlflow.log_metric("precision_micro", scores["precision_micro"])
        mlflow.log_metric("recall_micro", scores["recall_micro"])


        # --- DEFINITION EMPLACEMENT SOUHAITE DE SAUVEGARDE LOCALE DU MODELE
        path_model = f"models/logreg/logreg_{name}.joblib"
        # --- CREATION DU DOSSIER COMPLET S'IL N'EXISTE PAS ENCORE
        os.makedirs(os.path.dirname(path_model), exist_ok=True)
        # --- SAUVEGARDE DU MODELE EN LOCAL
        joblib.dump(scores["model"], path_model)
        # --- COPIE DANS mlruns/.../artifacts/
        mlflow.log_artifact(path_model)


        # 📥 Stockage dans le tableau de résultats
        df_results.append({
            "vecteur": name,
            "f1_micro": round(scores["f1_micro"], 3),
            "f1_macro": round(scores["f1_macro"], 3),
            "precision_micro": round(scores["precision_micro"], 3),
            "recall_micro": round(scores["recall_micro"], 3),
            "hamming_loss": round(scores["hamming_loss"], 4),
            "coverage_tags": round(scores["coverage_tags"], 4)
})


df_results = pd.DataFrame(df_results)
display(df_results.sort_values("f1_micro", ascending=False))


### 2.5. VALIDATION CROISEE SUR CHAQUE VECTEUR

In [ ]:
# ---------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# ---------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 6. VALIDATION CROISÉE SUR CHAQUE VECTEUR
# ---------------------------------------------------------
cv_results = []
# --- VALIDATION CROISEE POUR CHAQUE VECTEUR (AVEC PREPROCESSING SPECIFIQUE)
for name, X in X_dict.items():
    preprocess = preproc_dict.get(name)

    with mlflow.start_run(run_name=f"{model_type}_{name}_cv"):
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("vecteur", name)
        mlflow.log_param("preprocessing", preprocess)
        mlflow.log_param("validation", "cross_val")

        f1_cv = mdl.cross_validate_vector(
            name=name,
            X=X,
            Y=Y,
            model_class=model_class,
            model_wrapper=model_wrapper,
            preprocess=preprocess
        )

        mlflow.log_metric("f1_cv", f1_cv)

        cv_results.append({
            "vecteur": name,
            "f1_cv": f1_cv
        })

# --- AFFICHAGE DES RESULTATS
df_cv_results = pd.DataFrame(cv_results).sort_values("f1_cv", ascending=False)
display(df_cv_results)


### 2.6. COMPARAISON RESULTATS SPLIT VERSUS CROSS VALIDATION

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 7. MÉTRIQUES COMPARATIVES + DELTA CV/SPLIT
# ---------------------------------------------------------
model_metrics = []
# --- POUR CHAQUE VECTEUR ON RECUPERE LES RESULTATS D'ENTRAINEMENT SPLIT ET CV
for name in X_dict.keys():
    f1_split = df_results.query("vecteur == @name")["f1_micro"].values[0]
    f1_cv = df_cv_results.query("vecteur == @name")["f1_cv"].values[0]
    delta = round(f1_split - f1_cv, 3)
    hamming = df_results.query("vecteur == @name")["hamming_loss"].values[0]
    
    model_metrics.append({
        "vecteur": name,
        "F1_split": f1_split,
        "F1_cv": f1_cv,
        "delta_split_cv": delta,
        "hamming_loss": hamming
    })
# --- AFFICHAGE DES METRIQUES POUR CHAQUE VECTEUR/COMPARAISON RESULTATS SPLIT ET CROSS VALIDATION
df_model_metrics = pd.DataFrame(model_metrics).sort_values("F1_cv", ascending=False)
display(df_model_metrics)

### 2.7. SAUVEGARDE DU MEILLLEUR MODELE

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 8. SAUVEGARDE DU MEILLEUR MODÈLE
# ---------------------------------------------------------
best_vector_name = mdl.select_best_vecteur(
    df_results,
     min_f1=0.4,
     min_coverage=0.3,
     ranking_col="f1_micro"
     )
print(f"best_vector_name : {best_vector_name}")
# --- SELECTION DU MEILLEUR MODELE D'APRES LES METRIQUES PUIS SAUVEGARDE
# 📁 Créer le dossier complet s’il n’existe pas
model_directory = "models/logreg"
os.makedirs(model_directory, exist_ok=True)
final_model, best_vect_name, path = mdl.save_best_model(
    # df_model_metrics,
    df_results,
    splits_dict,
    y_train,
    model_class=model_class,
    model_wrapper=model_wrapper,
    model_type=model_type,
    save_dir=model_directory
)

### 2.8. FUSION DES MÉTRIQUES DANS UN TABLEAU COMPARATIF FINAL

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)



# ---------------------------------------------------------
# --- FUSION DES MÉTRIQUES DANS UN TABLEAU COMPARATIF FINAL
# ---------------------------------------------------------
# --- Fusionner les DataFrames sur la colonne 'vecteur'
df_merged = df_results.merge(df_cv_results, on="vecteur")

# ✨ Arrondi pour lisibilité
df_merged[["f1_micro", "hamming_loss", "coverage_tags", "f1_cv"]] = df_merged[
    ["f1_micro", "hamming_loss", "coverage_tags", "f1_cv"]
].round(3)

# ---------------------------------------------------------
# --- LOG DU TABLEAU COMPARATIF DANS MLFLOW
# ---------------------------------------------------------
os.makedirs(model_directory, exist_ok=True)
mlflow.end_run()
with mlflow.start_run(run_name="comparatif_vecteurs_final"):
    csv_path = f"{model_directory}/comparatif_vecteurs.csv"
    df_merged.to_csv(csv_path, index=False)
    mlflow.log_artifact(csv_path)

# ---------------------------------------------------------
# 👀 AFFICHAGE FINAL DANS LE NOTEBOOK
# ---------------------------------------------------------
display(df_merged.sort_values("f1_micro", ascending=False))




### 2.9 VISUALISATION DES SCORES

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# 📊 Barplot Seaborn logué dans MLflow
# ---------------------------------------------------------
img_model_directory = f"{model_directory}/mlflow_images"
os.makedirs(img_model_directory, exist_ok=True)

mlflow.end_run()
with mlflow.start_run(run_name="comparatif_f1_cv_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="f1_cv",
        title="Score F1 moyen par vecteur",
        save_path=f"{img_model_directory}/barplot_f1_cv.png"
    )

mlflow.end_run()
with mlflow.start_run(run_name="comparatif_coverage_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="coverage_tags",
        title="Couverture des tags par vecteur",
        save_path=f"{img_model_directory}/barplot_coverage_tags.png"
    )
mlflow.end_run()
with mlflow.start_run(run_name="comparatif_f1_micro_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="f1_micro",
        title="Score F1 (entraînement classique) par vecteur",
        save_path=f"{img_model_directory}/barplot_f1_micro.png"
    )

## ANNEXES

## 3. MODELE XGBOOST

### 3.1. CHARGEMENT DES FEATURES

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)

# -----------------------------------------
# --- 0. CONFIGURATION DU MODÈLE À TESTER
# -----------------------------------------
# ✅ Tu peux changer ces lignes pour benchmarker un autre modèle
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
model_class = XGBClassifier
model_wrapper = OneVsRestClassifier
model_type = "xgboost"


# ------------------------------
# --- 1. SELECTION DES FEATURES
# ------------------------------
full_df = pd.read_parquet("data/processed/full_explo_wo.parquet")
print(f"Dimensions du dataframe full_df : {full_df.shape}")
print(f" Colonnes du dataframe full_df : {full_df.columns.tolist()}")
print(full_df[["clean_title_body"]].head(3))

# --- A. CHARGEMENT DES VECTEURS
X_bow   = load_npz("models/bow/X_bow_full.npz")
X_tfidf = load_npz("models/tfidf/X_tfidf_full.npz")
X_svd   = np.load("models/svd/X_titlebody_svd10k.npy")
X_w2v   = np.load("models/word2vec/X_w2v_full.npy")
X_use   = np.load("models/use_model/embeddings_use_full.npy")
X_sbert = np.load("models/sbert/embeddings_sbert_full.npy")

print("# --- DIMENSIONS DES VECTEURS :")
for name, mat in [("BoW", X_bow), ("TF-IDF", X_tfidf), ("SVD", X_svd),
                  ("Word2Vec", X_w2v), ("USE", X_use), ("SBERT", X_sbert)]:
    print(f"# --- {name:<10}: {mat.shape}")


# --- B. CHARGEMENT DES LABELS MULTILABEL
mlb = joblib.load("models/tags/multilabel_binarizer_full.pkl")
Y_full = np.load("models/tags/y_tags_full.npy")

print(f"# --- Labels multilabel chargés : {Y_full.shape}")
print(f"# --- Nombre de tags avant filtrage : {len(mlb.classes_)}")

tag_counts = Y_full.sum(axis=0)
tag_mask = tag_counts >= 10  # seuil de rareté
Y_full_filtered = Y_full[:, tag_mask]

mlb_filtered = mlb
mlb_filtered.classes_ = np.array(mlb.classes_)[tag_mask]
print(f"# --- TAGS conservés après filtrage : {len(mlb_filtered.classes_)}")

### 3.2. DEFINITION DES VARIABLES VECTEURS A ENTRAINER

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ----------------------------------------------
# --- 2. SÉPARATION DES FEATURES ET DE LA CIBLE
# ----------------------------------------------
Y = Y_full_filtered
X_text = full_df["clean_title_body"]
print(f"# --- Matrice multilabel (Y) : {Y.shape}")
print(f"# --- Colonne textuelle (X) : {X_text.shape}")

# ---------------------------------------------------------
# --- 3. CRÉATION DES VECTEURS + DICTS DE SUPPORT
# ---------------------------------------------------------
X_dict = {
    "bow": X_bow,
    "tfidf": X_tfidf,
    "svd": X_svd,
    "w2v": X_w2v,
    "use": X_use,
    "sbert": X_sbert
}

preproc_dict = {
    "bow": None,
    "tfidf": None,
    "svd": None,
    "w2v": "scale",
    "use": "scale",
    "sbert": "scale"
}

### 3.3. DIVISION TRAIN / TEST POUR CHAQUE VECTEUR

In [ ]:
# ---------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# ---------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 4. DIVISION EN TRAIN / TEST SUR CHAQUE VECTEUR
# ---------------------------------------------------------
indices = np.arange(Y.shape[0])
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

splits_dict = {}
for name, X in X_dict.items():
    X_train, X_test, y_train, y_test = mdl.split_on_indices(X, Y, train_idx, test_idx)
    splits_dict[name] = (X_train, X_test)

print(f"# --- Splits prêts pour vecteurs : {list(splits_dict.keys())}")


### 3.4. ENTRAINEMENT DU MODELE POUR CHAQUE VECTEURS - RESULTATS

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
import mlflow
import mlflow.sklearn

df_results = []
uri = "file:///D:/machine_learning_training/openclassrooms_projects/05_categorisez_automatiquement_question/mlruns"
mlflow.set_tracking_uri(uri)

if mlflow.active_run():
    mlflow.end_run()
mlflow.set_experiment("xgboost_stackoverflow")



for name, (X_train, X_test) in splits_dict.items():
    preproc = preproc_dict.get(name)

    with mlflow.start_run(run_name=f"{model_type}_{name}"):
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("vecteur", name)
        mlflow.log_param("preprocessing", preproc)

        # 🧪 Entraînement + prédiction
        scores = mdl.train_and_score_vector_full_metrics(
            name=name,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            model_class=model_class,
            model_wrapper=model_wrapper,
            preprocess=preproc
        )

        # 📊 Logging des métriques
        mlflow.log_metric("f1_micro", scores["f1_micro"])
        mlflow.log_metric("hamming_loss", scores["hamming_loss"])
        mlflow.log_metric("coverage_score", scores["coverage_tags"])
        mlflow.log_metric("f1_macro", scores["f1_macro"])
        mlflow.log_metric("precision_micro", scores["precision_micro"])
        mlflow.log_metric("recall_micro", scores["recall_micro"])


        # 💾 Définir le chemin complet du modèle
        path_model = f"models/xgboost/xgboost_{name}.joblib"
        # 📁 Créer le dossier complet s’il n’existe pas
        os.makedirs(os.path.dirname(path_model), exist_ok=True)
        joblib.dump(scores["model"], path_model)
        mlflow.log_artifact(path_model)


        # 📥 Stockage dans le tableau de résultats
        df_results.append({
            "vecteur": name,
            "f1_micro": round(scores["f1_micro"], 3),
            "f1_macro": round(scores["f1_macro"], 3),
            "precision_micro": round(scores["precision_micro"], 3),
            "recall_micro": round(scores["recall_micro"], 3),
            "hamming_loss": round(scores["hamming_loss"], 4),
            "coverage_tags": round(scores["coverage_tags"], 4)
})


df_results = pd.DataFrame(df_results)
display(df_results.sort_values("f1_micro", ascending=False))


### 3.5. VALIDATION CROISEE SUR CHAQUE VECTEUR

In [ ]:
# ---------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# ---------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 6. VALIDATION CROISÉE SUR CHAQUE VECTEUR
# ---------------------------------------------------------
cv_results = []
# --- VALIDATION CROISEE POUR CHAQUE VECTEUR (AVEC PREPROCESSING SPECIFIQUE)
for name, X in X_dict.items():
    preprocess = preproc_dict.get(name)

    with mlflow.start_run(run_name=f"{model_type}_{name}_cv"):
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("vecteur", name)
        mlflow.log_param("preprocessing", preprocess)
        mlflow.log_param("validation", "cross_val")

        f1_cv = mdl.cross_validate_vector(
            name=name,
            X=X,
            Y=Y,
            model_class=model_class,
            model_wrapper=model_wrapper,
            preprocess=preprocess
        )

        mlflow.log_metric("f1_cv", f1_cv)

        cv_results.append({
            "vecteur": name,
            "f1_cv": f1_cv
        })

# --- AFFICHAGE DES RESULTATS
df_cv_results = pd.DataFrame(cv_results).sort_values("f1_cv", ascending=False)
display(df_cv_results)


### 3.6. COMPARAISON RESULTATS SPLIT VERSUS CROSS VALIDATION

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 7. MÉTRIQUES COMPARATIVES + DELTA CV/SPLIT
# ---------------------------------------------------------
model_metrics = []
# --- POUR CHAQUE VECTEUR ON RECUPERE LES RESULTATS D'ENTRAINEMENT SPLIT ET CV
for name in X_dict.keys():
    f1_split = df_results.query("vecteur == @name")["f1_micro"].values[0]
    f1_cv = df_cv_results.query("vecteur == @name")["f1_cv"].values[0]
    delta = round(f1_split - f1_cv, 3)
    hamming = df_results.query("vecteur == @name")["hamming_loss"].values[0]
    
    model_metrics.append({
        "vecteur": name,
        "F1_split": f1_split,
        "F1_cv": f1_cv,
        "delta_split_cv": delta,
        "hamming_loss": hamming
    })
# --- AFFICHAGE DES METRIQUES POUR CHAQUE VECTEUR/COMPARAISON RESULTATS SPLIT ET CROSS VALIDATION
df_model_metrics = pd.DataFrame(model_metrics).sort_values("F1_cv", ascending=False)
display(df_model_metrics)

### 3.7. SAUVEGARDE DU MEILLLEUR MODELE

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)
# ---------------------------------------------------------
# --- 8. SAUVEGARDE DU MEILLEUR MODÈLE
# ---------------------------------------------------------
# --- SELECTION DU MEILLEUR MODELE D'APRES LES METRIQUES PUIS SAUVEGARDE
# 📁 Créer le dossier complet s’il n’existe pas
model_directory = "models/xgboost"
os.makedirs(model_directory, exist_ok=True)
final_model, best_vect_name, path = mdl.save_best_model(
    df_model_metrics,
    splits_dict,
    y_train,
    model_class=model_class,
    model_wrapper=model_wrapper,
    model_type=model_type,
    save_dir=model_directory
)

### 3.8. FUSION DES MÉTRIQUES DANS UN TABLEAU COMPARATIF FINAL

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)



# ---------------------------------------------------------
# --- FUSION DES MÉTRIQUES DANS UN TABLEAU COMPARATIF FINAL
# ---------------------------------------------------------
# --- Fusionner les DataFrames sur la colonne 'vecteur'
df_merged = df_results.merge(df_cv_results, on="vecteur")

# ✨ Arrondi pour lisibilité
df_merged[["f1_micro", "hamming_loss", "coverage_tags", "f1_cv"]] = df_merged[
    ["f1_micro", "hamming_loss", "coverage_tags", "f1_cv"]
].round(3)

# ---------------------------------------------------------
# 📊 LOG DU TABLEAU COMPARATIF DANS MLFLOW
# ---------------------------------------------------------
os.makedirs("models/xgboost/", exist_ok=True)
mlflow.end_run()
with mlflow.start_run(run_name="comparatif_vecteurs_final"):
    csv_path = "models/comparatif_vecteurs.csv"
    df_merged.to_csv(csv_path, index=False)
    mlflow.log_artifact(csv_path)

# ---------------------------------------------------------
# 👀 AFFICHAGE FINAL DANS LE NOTEBOOK
# ---------------------------------------------------------
display(df_merged.sort_values("f1_micro", ascending=False))




### 3.9 VISUALISATION DES SCORES

In [ ]:
# -------------------------------------------
# --- RECHARGEMENT DES MODULES DE MODELISATION
# -------------------------------------------
import importlib
import src.modeling.modeling as mdl
importlib.reload(mdl)

# ---------------------------------------------------------
# -- BARPLOT SEABORN LOGUE DANS MLFLOW
# ---------------------------------------------------------
img_model_directory = f"{model_directory}/mlflow_images"
os.makedirs(img_model_directory, exist_ok=True)

mlflow.end_run()
with mlflow.start_run(run_name="comparatif_f1_cv_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="f1_cv",
        title="Score F1 moyen par vecteur",
        save_path=f"{img_model_directory}/barplot_f1_cv.png"
    )

mlflow.end_run()
with mlflow.start_run(run_name="comparatif_coverage_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="coverage_tags",
        title="Couverture des tags par vecteur",
        save_path=f"{img_model_directory}/barplot_coverage_tags.png"
    )

mlflow.end_run()
with mlflow.start_run(run_name="comparatif_f1_micro_barplot", nested=True):
    mdl.plot_and_log_barplot(
        df_scores=df_merged,
        metric="f1_micro",
        title="Score F1 (entraînement classique) par vecteur",
        save_path=f"{img_model_directory}/barplot_f1_micro.png"
    )